# Examine whether the significant mobility ties are weak

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime 
from dateutil import tz
import gzip
import time
import pickle
import copy


Bad key text.latex.unicode in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key text.latex.preview in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 125 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/jtl/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 157 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')


In [2]:
import sys
sys.path.append("../")
import utils

In [3]:
# read files
with open("../../data/02_intermediate/boston_stays.pickle", 'rb') as f:
    df_boston = pickle.load(f)

with open("../../data/02_intermediate/miami_stays.pickle", 'rb') as f:
    df_miami = pickle.load(f)

with open("../../data/02_intermediate/chicago_stays.pickle", 'rb') as f:
    df_chicago = pickle.load(f)

with open("../../data/03_processed/A_home_activity_three_cities_unweighted_dic.pickle", 'rb') as f:
    A_home_activity_unweighted_dic = pickle.load(f)
    
# with open("../../data/03_processed/greater_boston_ct_socioecon_shp.pickle", 'rb') as f:
#     df_shp = pickle.load(f)


In [4]:
# read evaluation files
with open("../../data/05_model_outputs/lasso_coefficients.pickle", 'rb') as f:
    lasso_coef = pickle.load(f)
    

In [5]:
lasso_coef.keys()

dict_keys(['boston', 'chicago', 'miami', 'total'])

In [25]:
lasso_coef['boston']['inc_median_household_2018']['lasso (no socio-demographics)'].index

Index(['Latin American', 'Caribbean', 'Brazilian', 'Fried Chicken',
       'Laundromat', 'Food Stand', 'Tennis Court', 'Fishing Store', 'Football',
       'Science Museum', 'Ski Area', 'Peking Duck', 'Cupcakes', 'Skating Rink',
       'French'],
      dtype='object')

# Significant ties are weak ties

In [6]:
# activity counts for the three cities
# boston
activity_counts_boston = np.unique(df_boston.cat, return_counts = True)

In [11]:
# miami
activity_counts_miami = np.unique(df_miami.cat, return_counts = True)

In [12]:
# chicago
activity_counts_chicago = np.unique(df_chicago.cat, return_counts = True)

In [13]:
# convert the counts to df
activity_counts_dic = {}
activity_counts_dic['boston']=activity_counts_boston
activity_counts_dic['miami']=activity_counts_miami
activity_counts_dic['chicago']=activity_counts_chicago

# turn them to dataframes
activity_counts_df_dic = {}
for key_ in activity_counts_dic.keys():
    activity_counts = activity_counts_dic[key_]
    activity_count_df = pd.DataFrame(activity_counts[1],
                                 index = activity_counts[0],
                                 columns = ['count'])
    sorted_activity_count_df = activity_count_df.sort_values('count', ascending=False)
    activity_counts_df_dic[key_] = sorted_activity_count_df


In [21]:
# compute the total
sorted_activity_count_total_df = activity_counts_df_dic['boston'].add(activity_counts_df_dic['miami'], fill_value=0.0).add(activity_counts_df_dic['chicago'], fill_value=0.0)
sorted_activity_count_total_df = sorted_activity_count_total_df.sort_values('count', ascending=False)
activity_counts_df_dic['total'] = sorted_activity_count_total_df


In [34]:
# use iterations to compute 1. total ratio, 2. top 20 strong mobility ties, 3. sorting of the significant mobility ties
weak_mobility_ties_results = {}
output_var_list = ['inc_median_household_2018', 'property_value_median_2018', 'rent_median_2018']
model_type = 'lasso (no socio-demographics)'

for output_var in output_var_list:
    print()
    print("---------")
    print(output_var)
    for key_ in activity_counts_df_dic.keys():
        print()
        print(key_)
        sig_activities = lasso_coef[key_][output_var][model_type].index
        ratio = activity_counts_df_dic[key_].loc[sig_activities].sum()[0]/activity_counts_df_dic[key_].sum()[0]
        print(ratio)
        print(activity_counts_df_dic[key_].sort_values('count', ascending=False).index[:15])
        print(sig_activities)

        ranking_list = []
        for sig_activity in sig_activities:
            ranking = list(activity_counts_df_dic[key_].index).index(sig_activity)
            ranking_list.append(ranking)
    #     print(sorted(ranking_list))
        print(ranking_list)



---------
inc_median_household_2018

boston
0.008992098345709094
Index(['Office', 'Residential', 'Building', 'Automotive', 'Church',
       'Salon / Barbershop', 'Doctor's Office', 'Pizza', 'American',
       'Gas Station', 'Gym', 'Road', 'Bank', 'Donuts', 'Hardware'],
      dtype='object')
Index(['Latin American', 'Caribbean', 'Brazilian', 'Fried Chicken',
       'Laundromat', 'Food Stand', 'Tennis Court', 'Fishing Store', 'Football',
       'Science Museum', 'Ski Area', 'Peking Duck', 'Cupcakes', 'Skating Rink',
       'French'],
      dtype='object')
[260, 190, 225, 304, 262, 540, 210, 527, 215, 319, 322, 557, 343, 114, 301]

miami
0.01959459759148913
Index(['Residential', 'Office', 'Building', 'Pool', 'Housing Development',
       'Automotive', 'Church', 'Doctor's Office', 'Grocery Store', 'Road',
       'Gym', 'Salon / Barbershop', 'Park', 'Gas Station', 'Fast Food'],
      dtype='object')
Index(['Discount Store', 'Pawn Shop', 'Storage', 'Supermarket', 'Concert Hall',
       'Mod